In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
# import wikipedia
# from newspaper import Article, ArticleException
# from GoogleNews import GoogleNews
import IPython
# from pyvis.network import Network

/home/pascal/miniconda3/envs/graph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

tokenizer_config.json: 100%|██████████| 1.23k/1.23k [00:00<00:00, 2.37MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 1.04MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 2.62MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 1.53MB/s]
special_tokens_map.json: 100%|██████████| 344/344 [00:00<00:00, 1.04MB/s]
config.json: 100%|██████████| 1.42k/1.42k [00:00<00:00, 2.27MB/s]
model.safetensors: 100%|██████████| 1.63G/1.63G [08:23<00:00, 3.23MB/s]


In [33]:
model.config

BartConfig {
  "_name_or_path": "Babelscape/rebel-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "b

In [8]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [26]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
            
    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [27]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 216,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }
    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

In [4]:
import pandas as pd

/tmp/ipykernel_8458/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
df = pd.read_csv('data/articles.csv')

In [11]:
df.head()

,url,article
0,https://thehimalayantimes.com/nepal/second-qua...,A 5.3 magnitude (ML) earthquake was felt acros...
1,https://thehimalayantimes.com/nepal/rural-muni...,The Commission for the Investigation of Abuse ...
2,https://thehimalayantimes.com/nepal/nea-cut-po...,Power supply to the Gautam Buddha Internationa...
3,https://thehimalayantimes.com/nepal/bagmati-pr...,Bagmati Chief Minister Rajendra Prasad Pandey ...
4,https://thehimalayantimes.com/nepal/10-persons...,Police arrested 10 people with 35 litres of re...


In [34]:
article = df.iloc[1, 1]
print(article)

The Commission for the Investigation of Abuse of Authority today filed a charge-sheet against 27 persons, including the chairperson of Likhu Tamakoshi Rural Municipality, Ramechhap, at the Special Court for embezzling around Rs 42.1 million during an electrification programme. Among those sued by the anti-graft body include rural municipality Chairperson Govinda Bahadur Khadka, Chief Administrative Officer Raju Prasad Neupane, accountant Durga Bahadur Dimdong, proprietor of Kathmandu-based Pike Construction Pvt Ltd., and 21 officials of various electrification users' committees. According to the CIAA, rural municipality chairperson Khadka in cahoots with other defendants had procured construction materials and equipment for electrification in the wards 1, 4 and 5 of the local level without inviting tenders. He unilaterally formed electrification users' committee at each ward to serve himself in misusing the state fund. It has also been found that Khadka did not even conduct a technical

In [31]:
kb = from_small_text_to_kb(article, verbose=True)

Num tokens: 341


In [32]:
kb.print()

Relations:
  {'head': 'Pike Construction Pvt Ltd', 'type': 'headquarters location', 'tail': 'Kathmandu'}
  {'head': 'Prevention of Corruption Act, 2002', 'type': 'point in time', 'tail': '2002'}
  {'head': 'Pike Construction', 'type': 'headquarters location', 'tail': 'Kathmandu'}


In [29]:
kb.print()

Relations:
  {'head': 'Belkotgadi', 'type': 'located in the administrative territorial entity', 'tail': 'Nuwakot District'}
  {'head': 'Patna', 'type': 'country', 'tail': 'India'}
  {'head': 'Nepal', 'type': 'contains administrative territorial entity', 'tail': 'Bagmati Province'}
  {'head': 'Bagmati Province', 'type': 'country', 'tail': 'Nepal'}


There seems to be loss of main information using this model

- First kb.print()
    - there is no information about embezzlement

- Sencod kn.print()
    - there is no information about earthquake

Model only goes for location or time, it seems!